In [1]:
!git clone https://github.com/ABaldrati/CLIP4Cir

Cloning into 'CLIP4Cir'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 53 (delta 14), reused 6 (delta 6), pack-reused 31 (from 1)
Receiving objects: 100% (53/53), 1.52 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
!pip install comet-ml==3.21.0
!pip install git+https://github.com/openai/CLIP.git
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f 'https://download.pytorch.org/whl/torch_stable.html'
!pip install pandas==1.4.2
!pip install urllib3==1.26.15
!pip install -U comet-ml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 6.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=66fb0a372174629ccf71487765fcac7bce15dbfca78e0b77b8d98b1f34f2b785
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0u9m3gr2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-0u9m3gr2
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:

In [4]:
import shutil
import os

# Define source and destination directories
source_dir = '/kaggle/input/fashioniq2'
destination_dir = '/kaggle/working/CLIP4Cir'

# Define the specific source directories for each folder
directories_to_copy = {
    'captions': 'captions-20220326T130604Z-001',
    'image_splits': 'image_splits-20220326T130551Z-001',
    'images': 'images'
}

# Loop through each directory and copy it
for folder_name, src_subdir in directories_to_copy.items():
    src_path = os.path.join(source_dir, src_subdir)
    dest_path = os.path.join(destination_dir, folder_name)
    
    # Ensure the destination directory exists
    os.makedirs(dest_path, exist_ok=True)
    
    # Copy the directory content recursively
    shutil.copytree(src_path, dest_path, dirs_exist_ok=True)

print("Directories copied successfully.")

Directories copied successfully.


In [5]:
import os
import shutil

# Define source and destination directories
source_dir = '/kaggle/working/CLIP4Cir'
destination_dir = os.path.join(source_dir, 'fashionIQ_dataset')

# Create FashionIQ_dataset directory if it does not exist
os.makedirs(destination_dir, exist_ok=True)

# Define the directories to move
directories_to_move = {
    'captions/captions': 'captions',
    'image_splits/image_splits': 'image_splits',
    'images/images': 'images'
}

# Loop through each directory and check if it exists before moving
for src_subdir, dest_subdir in directories_to_move.items():
    src_path = os.path.join(source_dir, src_subdir)
    dest_path = os.path.join(destination_dir, dest_subdir)
    
    # Check if the source directory exists
    if os.path.exists(src_path):
        # Move the directory to the new location
        shutil.move(src_path, dest_path)
        print(f"Moved {src_path} to {dest_path}")
    else:
        print(f"Source directory {src_path} does not exist.")

print("Directories moved successfully.")

Moved /kaggle/working/CLIP4Cir/captions/captions to /kaggle/working/CLIP4Cir/fashionIQ_dataset/captions
Moved /kaggle/working/CLIP4Cir/image_splits/image_splits to /kaggle/working/CLIP4Cir/fashionIQ_dataset/image_splits
Moved /kaggle/working/CLIP4Cir/images/images to /kaggle/working/CLIP4Cir/fashionIQ_dataset/images
Directories moved successfully.


In [6]:
from PIL import Image
import os

# Define the directory containing the images
images_dir = '/kaggle/working/CLIP4Cir/fashionIQ_dataset/images'

# Loop through all files in the directory
for filename in os.listdir(images_dir):
    if filename.endswith('.png'):  # Check if the file is a PNG image
        png_path = os.path.join(images_dir, filename)
        
        # Open the PNG image
        with Image.open(png_path) as img:
            # Convert the image to RGB (required for saving as JPG)
            rgb_img = img.convert('RGB')
            
            # Define the new file path with JPG extension
            jpg_path = os.path.join(images_dir, filename.replace('.png', '.jpg'))
            
            # Save the image as JPG
            rgb_img.save(jpg_path, 'JPEG')

            # Optionally, delete the original PNG file
            os.remove(png_path)

print("PNG files have been successfully converted to JPG.")

PNG files have been successfully converted to JPG.


In [7]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python /kaggle/working/CLIP4Cir/src/clip_fine_tune.py \
   --dataset FashionIQ \
   --api-key bKMwHTWGZ1fqpCdlAWHrPrP1M \
   --workspace longnguyenha050 \
   --experiment-name clip-finetune-fashioniq \
   --num-epochs 100 \
   --clip-model-name RN50x4 \
   --encoder both \
   --learning-rate 2e-6 \
   --batch-size 32 \
   --transform targetpad \
   --target-ratio 1.25  \
   --save-training \
   --save-best \
   --validation-frequency 1

Comet logging ENABLED
COMET INFO: Experiment is live on comet.com https://www.comet.com/longnguyenha050/fashioniq-clip-fine-tuning/224c67b453434eaba386c59143128f19

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
100%|███████████████████████████████████████| 402M/402M [00:05<00:00, 84.1MiB/s]
Both CLIP encoders will be fine-tuned
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Target pad with target_ratio = 1.25 preprocess pipeline is used
FashionIQ val - ['dress'] dataset in relative mode initialized
FashionIQ val - ['dress'] dataset in classic mode initialized
FashionIQ val - ['toptee'] dataset in relative mode initialized
FashionIQ val - ['toptee'] dataset in classic mode initialized
FashionIQ val - ['shi